In [2]:
import numpy as np
import pandas as pd
import networkx as nx
from scipy import stats as st
import matplotlib.pyplot as plt
G = nx.Graph()
# Given G as the graph

We use an undirected graph because human relationships are bi-directional in most cases. With an assumption that the author who comes first on a paper is the initiator of the relationship we would maybe have used a directed graph, but this is not a reasonable assumption.

In [ ]:
print(f'There are {G.number_of_nodes()} and {G.number_of_edges()} in the graph with resulting density of {nx.density(G)}')

In [ ]:
degrees = np.array([d for _, d in G.degree()])

print(f'Statistics for Degrees:\nMedian {degrees.median()}\nMean {degrees.mean()}\nMode {st.mode(degrees)}\nMin: {degrees.min()}\nMax: {degrees.max()}')

In [ ]:
strengths = []
for node in G.nodes:
    strengths.append(sum([G.get_edge_data(*edge)['weight'] for edge in G.edges(node)]))
print(f'Statistics for Node Strengths:\nMedian {strengths.median()}\nMean {strengths.mean()}\nMode {st.mode(strengths)}\nMin: {strengths.min()}\nMax: {strengths.max()}')

In [ ]:
plt.hist(degrees, nbins=100)
plt.show()

The following lines find the top author ID's by degree which we subsequently look up to find out what they study.

In [3]:
degrees = np.random.random(7)
G.nodes[np.argsort(degrees)[:5]]

array([0.20084418, 0.21006797, 0.28561496, 0.53535729, 0.63216144])

In [ ]:
spearman_cor = st.spearmanr(G.nodes['median_citation_count'], degrees)
plt.scatter(G.nodes['median_citation_count'], degrees)
plt.title(f'Median Citation Count vs Degree Count')
plt.xlabel(f'Median Citation Count')
plt.ylabel(f'Degree Count')
plt.show()

The spearman correlation s more robust to outliers, and it seems there are are couple of these in the data.